<a href="https://colab.research.google.com/github/jyp1111/TIL/blob/master/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pickle
import numpy as np
from collections import Counter
from os.path import isfile, join

Preprocessing

In [ ]:
# 전처리 과정 코드에 다 쓰고 싶었는데 학원 컴퓨터에 있는 듯 합니다.
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk import pos_tag

with open("wiki_00",mode="r",encoding="utf-8") as f:
    text=f.read()
textdf=pd.DataFrame(text.split("</doc>")[:-1])
textdf.columns=["content"]
textdf["content"]=textdf["content"].str.replace('<.*?>',"").str.replace("\n"," ")
textdf["tokenize"]=textdf["content"].apply(word_tokenize)
textdf["pos_tag"]=textdf["tokenize"].apply(pos_tag)

pos=["JJ","JJR", "JJS","NN","NNS","NNP","NNPS","VB","VBD","VBG","VBN","VBP","VBZ"]

textdf["reduced_postag"]=textdf["pos_tag"].apply(lambda ls: [v for v in ls if (v[1] in pos) and (len(v[0])>1)])
textdf["token_ls"]=textdf["reduced_postag"].apply(lambda ls: [v[0] for v in ls])

df=pd.DataFrame([",".join(token_ls) for token_ls in textdf["token_ls"].tolist()])

In [ ]:
class construction:
    def __init__(self,docs_ls_file,word_freq_file):

        if not isfile(join("./",docs_ls_file)):
            self.docs_ls_scv=pd.Series(dtype="object")
            self.word_freq=Counter()
            
        else:
            self.docs_ls_csv=pd.read_csv(docs_ls_file)["0"]    
            
            with open(word_freq_file,"rb") as f:
                self.word_freq=pickle.load(f)
        
        self.freq_ls=None
        self.index_word_dic=None
        self.word_index_dic=None
        
    def add_docs_ls(self,docs_ls,docs_ls_file,word_freq_file):
        concate_docs_ls=pd.concat([self.docs_ls_csv,docs_ls])
        self.word_freq+=Counter(sum(docs_ls.tolist(),[]))
        
        concate_docs_ls.to_csv(docs_ls_file,index=False)
        
        self.docs_ls_csv=pd.read_csv(docs_ls_file)["0"]
        with open(word_freq_file,'wb') as f:
            pickle.dump(self.word_freq, f)

    def get_word_freq_ls(self,vocab_size):
        word_ls,self.freq_ls=zip(*self.word_freq.most_common()[:vocab_size])
        self.index_word_dic=dict(enumerate(word_ls))
        self.word_index_dic={}
        for i,w in enumerate(word_ls):
            self.word_index_dic[w]=i
        with open("freq_ls.pickle","wb") as f:
            pickle.dump(self.freq_ls,f)
        with open("index_word_dic.pickle","wb") as f:
            pickle.dump(self.index_word_dic,f)
        with open("word_index_dic.pickle","wb") as f:
            pickle.dump(self.word_index_dic,f)
            
    def set_docs_ls(self):
        # 문서수 : 6041696
        for i in range(-(-6041696//10000)):
            with open(f"docs_ls/docs_ls{i}","wb") as f:
                pickle.dump(self.docs_ls_csv[i*10000:(i+1)*10000].str.replace("[","").str.replace("]","").str.replace("'","").str.replace(" ","").str.split(","), f)

class skip_gram:
    def __init__(self,docs_ls_file,window_size):
        with open("freq_ls.pickle","rb") as f:
            self.freq_ls=pickle.load(f)
        with open("index_word_dic.pickle","rb") as f:
            self.index_word_dic=pickle.load(f)
        with open("word_index_dic.pickle","rb") as f:
            self.word_index_dic=pickle.load(f)
        with open(docs_ls_file,"rb") as f:
            self.docs_ls=pickle.load(f)

        self.window_size=window_size
        if not isfile(join("./","coef.pickle")):
            self.coef_=None
        else:  
            with open("coef.pickle","rb") as f:
                self.coef_=pickle.load(f)        

    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def negative_sampling(self):
        n=len(self.word_index_dic)
        fre_ls=np.array(list(self.freq_ls))
        distribution=np.power(fre_ls,0.75)/np.power(fre_ls,0.75).sum()
        return np.random.choice(range(n), size=20, p=distribution)

    
    def connection(self,doc,i):
        result=set()
        for j,v in enumerate(doc):
            if v==self.index_word_dic[i]:
                result=result.union({self.word_index_dic[doc[k]] for k in range(max(j-self.window_size,0),min(j+self.window_size+1,len(doc))) if doc[k] in self.word_index_dic})
        return result-{i}            

    def train(self,vec_dim,learning_rate):
        n=len(self.word_index_dic)
        m=len(self.docs_ls)
        
        if self.coef_==None:
            W_1=np.random.rand(n,vec_dim)
            W_2=np.random.rand(n,vec_dim)
        else: 
            W_1,W_2=self.coef_

        for i in range(n):
            return_set=set().union(*self.docs_ls.apply(self.connection,args=(i,)))

            neighbor=sorted(list(return_set))
            negative_words=self.negative_sampling()

            positive=np.ones(len(neighbor))
            negative=np.zeros(20)

            y=np.concatenate((positive,negative))
            relative_indices=neighbor+list(negative_words)
            hyp=self.sigmoid(np.dot(W_2[relative_indices],W_1[i]))
            loss=-(y*np.log(hyp)+(1-y)*np.log(1-hyp)).mean()

            first_par=W_1[i]
            W_1[i]-=np.dot(W_2[relative_indices].T,hyp-y)*learning_rate/len(y)
            W_2[relative_indices]-=np.dot((hyp-y).reshape(-1,1),first_par.reshape(1,-1))*learning_rate/len(y)
            if i%100==0:
                print(i)
        self.coef_=(W_1,W_2)
        with open("coef.pickle","wb") as f:
            pickle.dump(self.coef_,f)
        
    def check(self,word1,word2,word3):
        word_matrix=self.coef_[0]
        
        word1_index=self.word_index_dic[word1]
        word2_index=self.word_index_dic[word2]
        word3_index=self.word_index_dic[word3]
        result_vec=word_matrix[word1_index]-word_matrix[word2_index]+word_matrix[word3_index]
        
        return self.index_word_dic[np.argmin(np.multiply(word_matrix-result_vec,word_matrix-result_vec).sum(axis=1))]


Construction

In [ ]:
import csv
from os import listdir
from os.path import isfile, join, isdir
csv_files=["./text_csv/"+f  for f in listdir("./text_csv/")  if isfile(join("./text_csv/", f))]
csv_files.sort()

In [ ]:
len(csv_files)

139

In [ ]:
for x in csv_files:
    df=pd.read_csv(x)
    df=df.dropna()
    df["0"]=df["0"].apply(lambda v : ",".join(v.split(",")[1:]))
    df=df.dropna()
    df["0"].to_csv(x,index=False)

In [ ]:
wiki=construction("docs_ls.csv","freq.pickle")
for file in csv_files[135:]:
    df=pd.read_csv(file)
    wiki.add_docs_ls(df["0"].str.split(","),"docs_ls.csv","freq.pickle")
    print(file)

./text_csv/FF.csv
./text_csv/FG.csv
./text_csv/FH.csv
./text_csv/FI.csv


In [ ]:
wiki=construction("docs_ls.csv","freq.pickle")
wiki.get_word_freq_ls(30000)
wiki.set_docs_ls()

In [ ]:
from os import listdir
from os.path import isfile, join, isdir
docs_ls_files=["./docs_ls/"+f  for f in listdir("./docs_ls/")  if isfile(join("./docs_ls/", f))]
docs_ls_files.sort(key=lambda x:int(x[17:]))

In [ ]:
len(docs_ls_files)

605

train

In [ ]:
model=skip_gram(docs_ls_files[0],1)
model.train(3,0.01)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [ ]:
model.check("Athens","Greece","Korea")

'thwarted'

In [ ]:
for docs_ls_file in docs_ls_files:
    model=skip_gram(docs_ls_file,1)
    model.train(3,0.01)
model.check("Athens","Greece","Korea")